In [5]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random, numpy as np

torch.manual_seed(1)

In [4]:
torch.DoubleTensor([1])


 1
[torch.DoubleTensor of size 1]

In [23]:
#emb experiment
emb = nn.Embedding(100, 100)
emb('my name is uddeshya')

AttributeError: 'str' object has no attribute 'dim'

In [15]:
#some random semantics test data
num_data = 100
dataX = [autograd.Variable(torch.randn((1, 75))) for _ in range(num_data)]
dataY = [autograd.Variable(torch.DoubleTensor([random.randint(1,11)])) for _ in range(num_data)]
# print(dataY)
# print(dataX)

In [9]:
inp_dim=75
hidden_dim=75
lstm = nn.LSTM(inp_dim, hidden_dim) # Input dim is 3, output dim is 3
inputs = [autograd.Variable(torch.randn((1, inp_dim)))
          for _ in range(10)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (autograd.Variable(torch.randn(1, 1, hidden_dim)),
          autograd.Variable(torch.randn((1, 1, hidden_dim))))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (autograd.Variable(torch.randn(1, 1, hidden_dim)),
          autograd.Variable(torch.randn((1, 1, hidden_dim))))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.4422 -0.0836 -0.3623  0.0170  0.0153  0.3019  0.9204  0.0295  0.2799

Columns 9 to 17 
   0.0171  0.3507 -0.2216 -0.4838 -0.0374  0.1170  0.3147  0.2965 -0.6561

Columns 18 to 26 
  -0.0643  0.7508 -0.4947 -0.1613 -0.0253  0.2302  0.1747  0.4461  0.1309

Columns 27 to 35 
   0.0815 -0.1382 -0.5025  0.0445 -0.0946 -0.1964  0.1771 -0.2011  0.1233

Columns 36 to 44 
   0.1657 -0.1282 -0.0748 -0.3011  0.1242 -0.0762  0.3110 -0.0740  0.3555

Columns 45 to 53 
  -0.4821 -0.4017  0.0443  0.3157  0.5549 -0.2614  0.0037  0.2796  0.0886

Columns 54 to 62 
  -0.3237 -0.0264 -0.0998 -0.1732 -0.2605 -0.0618  0.2819 -0.0445  0.1246

Columns 63 to 71 
   0.1765  0.1309  0.1503  0.1623 -0.2204 -0.1555 -0.4385  0.1237  0.0111

Columns 72 to 74 
  -0.1999  0.1897 -0.5549

(1 ,.,.) = 

Columns 0 to 8 
   0.0849  0.0138 -0.5418 -0.1290 -0.1049  0.4493  0.2657  0.0803  0.2891

Columns 9 to 17 
   0.1018  0.0534  0.0810 -0.4659 -0.0364 -0.1796  0.2171 -

In [16]:
#action LSTM

class LSTMClassifier(nn.Module):
    def __init__(self, joints_dim, hidden_dim, label_size, batch_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.lstm = nn.LSTM(joints_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

    def forward(self, seq_of_vec):
#         embeds = self.word_embeddings(sentence)
#         x = embeds.view(len(sentence), self.batch_size , -1)
        x = torch.cat(seq_of_vec).view(len(seq_of_vec), self.batch_size, -1)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y  = self.hidden2label(lstm_out[-1])
        log_probs = F.log_softmax(y)
        return log_probs

In [24]:
model0 = LSTMClassifier(75, 75, 10, 1)

In [20]:
x = torch.cat(dataX)
len(x)

100

In [25]:
model0.forward(dataX)

/home/udion/anaconda3/envs/DeepCV3.5/lib/python3.5/site-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Variable containing:
-2.3174 -2.3126 -2.3362 -2.2584 -2.3638 -2.2537 -2.2875 -2.4089 -2.2500 -2.2505
[torch.FloatTensor of size 1x10]